## Download glove embedding

In [ ]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt

--2021-09-02 06:56:17--  https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53933315 (51M) [text/plain]
Saving to: ‘glove.840B.300d.sst.txt’

glove.840B.300d.sst 100%[===================>]  51.43M   176MB/s    in 0.3s    

2021-09-02 06:56:19 (176 MB/s) - ‘glove.840B.300d.sst.txt’ saved [53933315/53933315]



In [ ]:
!cp glove.840B.300d.sst.txt sample_data/glove.840B.300d.sst.txt

## Importing necessary packages

In [3]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import itertools
from collections import Counter
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import *
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import array
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import sequence
from keras.regularizers import *
import tensorflow as tf
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
import timeit

## Supporting Functions

In [4]:
def dumpPickle(path,object_to_dump):
	f=open(path,'wb')
	pickle.dump(object_to_dump,f)
	f.close()

#Reads an object stores at 'path'
def readPickle(path):
	f=open(path,'rb')
	ob_to_load=pickle.load(f)
	f.close()
	return ob_to_load

#Loads an embedding matrix stores at emb_pickle_path if doesn't exists a pickled file, then creates One which is initialized by normal distribution with mean and std dev of the embeddings used, if given the option to save it saves
#the newly created embedding matrix at saveName
def load_create_embedding_matrix(word_index,vocab_size,emb_dim,emb_path,emb_pickle_path=False,save=False,saveName=None):
	if not emb_pickle_path:
		embedding_dict={}
		f=open(emb_path,'rb')
		for line in f:
			fields=line.split()
			word=fields[0]
			w_e=np.asarray(fields[1:],dtype='float32')
			embedding_dict[word]=w_e
		f.close()
		allembs=np.stack(embedding_dict.values())
		emb_mean,emb_std=allembs.mean(),allembs.std()
		embedding_matrix=np.random.normal(emb_mean,emb_std,(vocab_size,emb_dim))
		for word,index in word_index.items():
			#print(word)
			vector=embedding_dict.get(word)
			if vector is not None:
				embedding_matrix[index]=vector
		if save:
			dumpPickle(saveName,embedding_matrix)
	else:
		f=open(emb_pickle_path,'rb')
		embedding_matrix=pickle.load(f)
		f.close()
	return embedding_matrix


#creates a tokenizer from training data file in csv format, if there is one it loads and returns
def load_create_tokenizer(train_data,tok_path=None,savetokenizer=False):
	if tok_path == None:
		tokenizer=Tokenizer()
		tokenizer.fit_on_texts(train_data)
		len(tokenizer.word_index)
		if savetokenizer:
			dumpPickle('./New_Tokenizer.tkn',tokenizer)
	else:
		tokenizer=readPickle(tok_path)
	return tokenizer

def load_create_padded_data(X_train,savetokenizer,padPath=None,isPaddingDone=False,maxlen=None,tokenizer_path=None,save_new_padded_data=False,path_for_new_data=False):
	#print(tokenizer_path)
	if not isPaddingDone:
		tokenizer=load_create_tokenizer(X_train,tok_path=tokenizer_path,savetokenizer=savetokenizer)
		#word_index=tokenizer.word_index
		X_train=tokenizer.texts_to_sequences(X_train)
		X_train=pad_sequences(X_train,maxlen=maxlen)
		if save_new_padded_data:
			dumpPickle(path_for_new_data,X_train)
	else:
		X_train=readPickle(padPath)
	return X_train

## loading dataset

In [6]:
data = pd.read_csv('/content/TWEET_DATA_IEEE_TCSS_-_Sheet1.csv') 
data.head()

,RT @Bipartisanism: Katie Couric just schooled #TedCruz on gay marriage in epic fashion. http://t.co/JtNOPqqOug,STM
0,RT @stevesilberman: If you missed it: Sick bur...,EXP
1,@jk_rowling & the Never-Ending Story: With a #...,QUE
2,RT @somebadideas: The emotional & personal eff...,EXP
3,RT @jk_rowling: A 9-year-old Nigerian girl has...,STM
4,RT @jk_rowling: I don't want to say too much m...,EXP


## Splitting in 80-20 ratio for test and train 

In [7]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, test_size=0.2) 
train=pd.DataFrame(train)
train.columns=["Tweet","Label"]
test=pd.DataFrame(test)
test.columns=["Tweet","Label"]

In [8]:
train_Text=train["Tweet"]
train_label=train["Label"]
test_Text=test["Tweet"]
test_label=test["Label"]

train = [s.strip() for s in train_Text]
test = [s.strip() for s in test_Text]

text = train + test
max1=0

#making same length of all text 
for i in range(0,len(text)):
	data=text[i].split(" ")
	max2=len(data)
	if(max2>max1):
		max1=max2


sequence_length = max1

## Creating padded data and embedding matrix

In [9]:
print("creating data")


tokenizer=load_create_tokenizer(train,None,True)
X_train=load_create_padded_data(X_train=train,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
X_test=load_create_padded_data(X_train=test,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
word_index=tokenizer.word_index
embedding_matrix=load_create_embedding_matrix(word_index,len(word_index)+1,300,'./glove.840B.300d.sst.txt',False,True,'./Emb_Mat.mat')

print("data created")

creating data


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


data created


## One-hot encode

In [10]:
lbl_dict={}
index=0
for dial_lbls in train_label:
	if dial_lbls not in lbl_dict:
		lbl_dict[dial_lbls]=index
		index=index+1
print((lbl_dict))

def create_label(label):
	
    Y=[]
    for i in label:
    	xxx=np.zeros(len(lbl_dict)) 
    	j=lbl_dict.get(i)
    	xxx[j]=1
    	Y.append(xxx)
    return Y

label = train_label
Y_train = create_label(label)

label = test_label
Y_test = create_label(label)

y_train=np.array([i for i in Y_train])
y_test=np.array([i for i in Y_test])

embedding_dim = 300

{'STM': 0, 'OTH': 1, 'EXP': 2, 'THT': 3, 'SUG': 4, 'QUE': 5, 'REQ': 6}


In [11]:
X_train.shape

(5999, 57)

In [12]:
X_test.shape

(1500, 57)

In [13]:
embedding_matrix.shape

(15188, 300)

## Creating CNN 1-D Model

In [17]:
from tensorflow.keras.optimizers import Adam
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=len(word_index)+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=sequence_length)(inputs)
conv_0 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(embedding)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)
dropout = Dropout(0.1)(maxpool_0)
conv_1 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(dropout)
maxpool_1 = MaxPooling1D(pool_size=2)(conv_1)
flatten = Flatten()(maxpool_1)
predictions = Dense(1000, activation='relu')(flatten)
#predictions1 = output layer
#Lets assume, correct output => [0,1,0,0,0,0,0,0]; generated output => [0.03, 0.90, 0.01, 0.02, 0.01, 0.01, 0.01, 0.01]
predictions1 = Dense(len(lbl_dict), activation='softmax')(predictions)
adam = Adam(lr=0.01, decay=0.3)
model = Model(inputs=inputs, outputs=predictions1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_test, y_test))


predicted=model.predict(X_test)
#print("DONE")

Creating Model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/4
94/94 [==============================] - 9s 87ms/step - loss: 1.5672 - accuracy: 0.5174 - val_loss: 1.3645 - val_accuracy: 0.5293
Epoch 2/4
94/94 [==============================] - 8s 84ms/step - loss: 1.3482 - accuracy: 0.5304 - val_loss: 1.3414 - val_accuracy: 0.5387
Epoch 3/4
94/94 [==============================] - 8s 84ms/step - loss: 1.3212 - accuracy: 0.5429 - val_loss: 1.3245 - val_accuracy: 0.5473
Epoch 4/4
94/94 [==============================] - 8s 84ms/step - loss: 1.3012 - accuracy: 0.5528 - val_loss: 1.3147 - val_accuracy: 0.5513


In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 57)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 57, 300)           4556400   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 57, 32)            28832     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 28, 32)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 28, 32)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 28, 32)            3104      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 14, 32)            0     

## Evaluate CNN Model

In [19]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_predict
# from sklearn.model_selection import train_test_split

print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.5513333333333333
              precision    recall  f1-score   support

           0     0.5000    0.1555    0.2372       328
           1     0.0000    0.0000    0.0000        76
           2     0.5551    0.9773    0.7080       794
           3     0.0000    0.0000    0.0000        69
           4     0.0000    0.0000    0.0000        70
           5     0.0000    0.0000    0.0000       120
           6     0.0000    0.0000    0.0000        43

    accuracy                         0.5513      1500
   macro avg     0.1507    0.1618    0.1350      1500
weighted avg     0.4032    0.5513    0.4267      1500

[[ 51   0 277   0   0   0   0]
 [  3   0  73   0   0   0   0]
 [ 18   0 776   0   0   0   0]
 [ 14   0  55   0   0   0   0]
 [  8   0  62   0   0   0   0]
 [  4   0 116   0   0   0   0]
 [  4   0  39   0   0   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Bi-LSTM Model

In [20]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  
maxlen=200

inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(max_features, 72)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 72)          1440000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         70144     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 1,609,089
Trainable params: 1,609,089
Non-trainable params: 0
_________________________________________________________________


## Evaluate Bi-LSTM Model

In [21]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
    
    
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_predict
# from sklearn.model_selection import train_test_split

print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.5513333333333333
              precision    recall  f1-score   support

           0     0.5000    0.1555    0.2372       328
           1     0.0000    0.0000    0.0000        76
           2     0.5551    0.9773    0.7080       794
           3     0.0000    0.0000    0.0000        69
           4     0.0000    0.0000    0.0000        70
           5     0.0000    0.0000    0.0000       120
           6     0.0000    0.0000    0.0000        43

    accuracy                         0.5513      1500
   macro avg     0.1507    0.1618    0.1350      1500
weighted avg     0.4032    0.5513    0.4267      1500

[[ 51   0 277   0   0   0   0]
 [  3   0  73   0   0   0   0]
 [ 18   0 776   0   0   0   0]
 [ 14   0  55   0   0   0   0]
 [  8   0  62   0   0   0   0]
 [  4   0 116   0   0   0   0]
 [  4   0  39   0   0   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Incorrect samples from the test set and its reasons.

1. RT @nokatieno: because in Afghanistan a 13 year old girl was shot in    the head for going to school #YesAllWomen 
       GIVEN: THT     CORRECTION: STM
  
2. RT @schemaly: Why do girls always fo@to the bathroom together?" #YesAllWomen are taught necessary buddy systems as girls and are then made
       GIVEN: EXP     CORRECTION: STM

3. RT @FloraWest: Thought it was time to bring this back: #YesAllWomen http://t.co/oll0yeMQxK
       GIVEN: EXP     CORRECTION: THT


# Reasons for misclassification

1. Synonyms can lead to issues similar to contextual understanding because we use many different words to express the same idea.

2. Informal phrases, expressions, idioms, and culture-specific lingo present in various tweets which also changes the meaning of sentence.